# Optimizing Model Parameters

지금까지 모델 만들었고, 데이터 불러왔으니 학습을 시켜보자
우리 데이터로 파라메터 최적화를 하여 모델을 입증하고 테스트하는 거야
모델을 학습하는 건 반복적인 처리인데, 각 반복에서 모델은 출력에 대한 예측을 하고 
예측에 대한 오차(loss)를 계산해, 오차를 각 파라미터를 기준 변수로 해서 미분한 값들을 모은다.
그리고 이 파라미터들을 경사 하강법 이용하여 최적화한다.


## Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100.0%
100.0%
100.0%
100.0%


## Hyperparameters

하이퍼파라미터를 조정해서 모델 학습이나 수렴률(얼마나 빨리 수렴에 가까워지는가, 수치해석, 최적화에서 오차가 반복마다 얼마나 빠르게 줄어드는지)에 영향을 줄 수 있다.

학습을 위해 정하게 되는 파라미터들
- Number of Epochs: 반복횟수
- Batch Size: 파라미터 업데이트 전, 한 번에 네트워크에 통과(전파) 시키는 데이터 샘플의 개수
- Learning Rate: 각 배치/에폭마다 모델 파라미터를 얼마나 크게 업데이트할지 정하는 값, 값이 작으면 학습이 느려지고, 너무 크면 학습이 불안정해짐

learning_rate = 1e-3 (= 0.001): 업데이트할 때 한 번에 움직이는 **보폭(step size)**이 0.001 정도라는 의미다. 보통 경사하강법 계열 업데이트가 
θ
←
θ
−
η
∇
θ
L
θ←θ−η∇ 
θ
 L 형태인데, 여기서 
η
η가 learning rate야

e가 지수승을 나타냄

학습률이 너무 작으면: 업데이트가 너무 조금씩이라 학습이 느리거나 덜 된다.
​

학습률이 너무 크면: 최솟값 근처에서 튕기거나 발산해서 학습이 불안정해질 수 있다.


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop

하이퍼파라미터를 정하면, 우리는 최적화 루프를 가지고 모델을 학습하고 최적화할 수 있다. 
각 최적화 루프의 반복은 에폭(epoch)라고 부른다.

각 에폭은 두가지 부분으로 구성됨
- The Train Loop: 학습 데이터셋을 반복하며, 손실을 줄이도록 파라미터를 업데이트해 최적값애 수렴하도록 학습
- The Validation/Test Loop: (검증/테스트) 데이터셋을 반복하며, 파라미터 업데이트 없이 성능을 평가해 모델이 개선되고 있는지 확인

## Loss Function

훈련 데이터가 주어졌을 때, 학습되지 않은 네트워크는 정답을 내지 못할 가능성이 크다. 손실 함수는 모델이 얻은 결과가 목표값과 얼마나 다른지의 정도를 측정하며, 학습 과정에서 손실 함수를 최소화하고자 하는 값이 바로 손실이다. 손실을 계산하기 위해, 주어진 데이터 샘플의 입력값으로 예측을 만든 다음 그 예측값을 실제 정답 라벨 값과 비교한다.

기본적인 손실 함수는 회귀(regression) 작업에서 nn.MSELoss(평균제곱오차, Mean Square Error)를, 분류(classification)작업에서는 nn.NLLLose(음의 로그우도, Negative Log Likelihood)를 대표적인 손실 함수로 사용

우리 모델의 아웃풋 로짓(각 클래스가 얼마나 그럴듯한지)를 nn.CrossEntrophyLoss에 통과시킴, 그리고 그것은 로짓을 정규화하고 예측 오차를 계산한다.

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer

모델 오차를 각 트레이닝 단계에서 줄이도록 모델 파라미터를 조정하는 것이 최적화

최적화 알고리즘은 어떻게 이 과정이 수행되는지 결정한다.(in this example we use Stochastic Gradient Descent)

모든 최적화 로직은 optimizer 객체 않에 캡술화되어있다.

SGD optimizer를 사용할건데, 추가로 많은 다른 optimizer들이 있다. (ADAM, RMSProp)

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 루프에서는 3 단게 동안 최적화가 일어나는데

- optimizer.zero_frad() 호출하여 모델 파라미터의 기울기를 초기화, double-counting을 예방, 명시적으로 각 반복에서 경사를 0으로 만듦

- loss.backword()를 호출하여 예측 손실을 역전파함, 파이토치가 loss를 각 파라미터에 대해 미분한 gradient를 각 파라미터 객체에 저장해 둔다.

- 일단 gradient를 얻고, optimizer.step()을 호출하여 backword pass에서 수집된 gradient에 의해 파라미터를 결정

## Full Implementation

train_loop(): 최적화
test_loop(): 검증 데이터로 모델 성능 평가

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

모델 실행

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.313030  [   64/60000]
loss: 2.295364  [ 6464/60000]
loss: 2.277286  [12864/60000]
loss: 2.270979  [19264/60000]
loss: 2.256611  [25664/60000]
loss: 2.239195  [32064/60000]
loss: 2.239461  [38464/60000]
loss: 2.210912  [44864/60000]
loss: 2.215934  [51264/60000]
loss: 2.184968  [57664/60000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 2.180083 

Epoch 2
-------------------------------
loss: 2.192493  [   64/60000]
loss: 2.180298  [ 6464/60000]
loss: 2.129351  [12864/60000]
loss: 2.143706  [19264/60000]
loss: 2.097535  [25664/60000]
loss: 2.048574  [32064/60000]
loss: 2.065143  [38464/60000]
loss: 1.994414  [44864/60000]
loss: 2.009582  [51264/60000]
loss: 1.936782  [57664/60000]
Test Error: 
 Accuracy: 56.3%, Avg loss: 1.939402 

Epoch 3
-------------------------------
loss: 1.966805  [   64/60000]
loss: 1.938055  [ 6464/60000]
loss: 1.832011  [12864/60000]
loss: 1.873577  [19264/60000]
loss: 1.762809  [25664/60000]
loss: 1.712655  [32064/600

에폭을 거듭할수록 정확도가 상승하고 평균 loss가 하강 -> 학습이 진행되며 성능이 개선되고 있음